In [19]:
import face_recognition
import cv2
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn import svm
from sklearn.covariance import EllipticEnvelope
import pickle

In [20]:
#Generate embedding array of known faces using 5 images per face
embedding_array = []
def append_array(name,embedding_array):
    image = []
    face_encoding = []
    for i in range(5):
        image.append(face_recognition.load_image_file("data_images/"+name+"/"+name+str(i)+".jpg"))
        face_encoding.append(face_recognition.face_encodings(image[i-1])[0])
        embedding_array.append(list(face_encoding[i-1]))

    
    return embedding_array

embedding_array = append_array('Aditya',embedding_array)
embedding_array = append_array('Joshua',embedding_array)
embedding_array = append_array('Carrisa',embedding_array)
embedding_array = append_array('Madeline',embedding_array)
embedding_array = append_array('Will',embedding_array)

In [98]:
#Generate embedding array of known faces using 1 image per face but add noise
embedding_array = []
def append_array(name,embedding_array):
    image=face_recognition.load_image_file("images/"+name+".jpg")
    face_encoding=(face_recognition.face_encodings(image)[0])
    embedding_array.append(list(face_encoding))
    #Add noise
    for j in range(29):
        noisy_image = face_encoding + np.random.normal(0.00001)
        embedding_array.append(list(noisy_image))

    return embedding_array

embedding_array = append_array('Aditya',embedding_array)
embedding_array = append_array('Joshua',embedding_array)
embedding_array = append_array('Carrisa',embedding_array)
embedding_array = append_array('Madeline',embedding_array)

In [21]:
#Total number of embeddings
len(embedding_array)

25

In [22]:
#Load testing embeddings

# Load a sample picture and learn how to recognize it.
Aditya10_image = face_recognition.load_image_file("images/Aditya.jpg")
Aditya10_face_encoding = face_recognition.face_encodings(Aditya10_image)[0]

# Load a third sample picture and learn how to recognize it.
Esti_image = face_recognition.load_image_file("images/Esti.jpg")
Esti_face_encoding = face_recognition.face_encodings(Esti_image)[0]

# Load a fourth sample picture and learn how to recognize it.
Will_image = face_recognition.load_image_file("images/Will.jpg")
Will_face_encoding = face_recognition.face_encodings(Will_image)[0]

# Load a fourth sample picture and learn how to recognize it.
Carr_image = face_recognition.load_image_file("images/Carrisa.jpg")
Carr_face_encoding = face_recognition.face_encodings(Carr_image)[0]

# Load a fourth sample picture and learn how to recognize it.
random1_image = face_recognition.load_image_file("random_faces/random1.jpg")
random1_face_encoding = face_recognition.face_encodings(random1_image)[0]

# Load a fourth sample picture and learn how to recognize it.
random2_image = face_recognition.load_image_file("random_faces/random2.jpg")
random2_face_encoding = face_recognition.face_encodings(random2_image)[0]

# Load a fourth sample picture and learn how to recognize it.
random3_image = face_recognition.load_image_file("random_faces/random3.jpg")
random3_face_encoding = face_recognition.face_encodings(random3_image)[0]


In [23]:
#Generate numpy array for the embeddings
X = np.array(embedding_array)

In [24]:
#Using SVM
clf = svm.OneClassSVM(nu=0.2, kernel="rbf", gamma=0.0078125)
clf.fit(X)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.0078125,
      kernel='rbf', max_iter=-1, nu=0.2, random_state=None, shrinking=True,
      tol=0.001, verbose=False)

In [25]:
#Pickle the svm model
file_Name = "./models/svm_model"
fileObject = open(file_Name,'wb') 
pickle.dump(clf,fileObject)   
fileObject.close()

with open(file_Name, 'rb') as pickle_file:
    clf = pickle.load(pickle_file)

In [26]:
#Using Isolation Forrest
clf = IsolationForest(n_estimators=5, max_samples=5, max_features=128,behaviour='new',contamination='auto')
clf.fit(X)

TypeError: __init__() got an unexpected keyword argument 'behaviour'

In [133]:
#Pickle the isolation forest model
file_Name = "./models/iso_forrest_model"
fileObject = open(file_Name,'wb') 
pickle.dump(clf,fileObject)   
fileObject.close()

with open(file_Name, 'rb') as pickle_file:
    clf = pickle.load(pickle_file)

In [13]:
#Using Robust Covariance
clf = EllipticEnvelope(random_state=0, contamination=0.03846)
clf.fit(X)

'''
Notes:
Outlier detection from covariance estimation may break or not perform well in high-dimensional settings. 
In particular, one will always take care to work with n_samples > n_features ** 2
'''

/anaconda3/lib/python3.6/site-packages/sklearn/covariance/robust_covariance.py:622: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


'\nNotes:\nOutlier detection from covariance estimation may break or not perform well in high-dimensional settings. \nIn particular, one will always take care to work with n_samples > n_features ** 2\n'

In [14]:
#Pickle the robust covariance model
file_Name = "./models/rob_covariance_model"
fileObject = open(file_Name,'wb') 
pickle.dump(clf,fileObject)   
fileObject.close()

with open(file_Name, 'rb') as pickle_file:
    clf = pickle.load(pickle_file)

In [15]:
#Check with random pictures
clf.predict([random3_face_encoding])

array([1])

In [146]:
clf.predict([random2_face_encoding])

array([-1])

In [147]:
clf.predict([random1_face_encoding])

array([-1])

In [148]:
clf.predict([random3_face_encoding])

array([-1])

In [149]:
clf.predict([Aditya10_face_encoding])

array([1])

In [150]:
clf.predict([Esti_face_encoding])

array([-1])

In [151]:
clf.predict([Will_face_encoding])

array([-1])

In [152]:
clf.predict([Carr_face_encoding])

array([1])